In [2]:
!pip install -q tensorflow numpy pandas nltk keras
print('done!')

done!


In [3]:
import pandas as pd
import copy

# importa dataset
mental_health_conversation_dataset = pd.read_csv('./datasets/Mental_Health_FAQ.csv')
mental_health_conversation_dataset.drop(columns=['Question_ID'], axis=0, inplace=True)
# importa dataset
QA_python_dataset = pd.read_csv('./datasets/Dataset_Python_Question_Answer.csv')
QA_python_dataset = QA_python_dataset.rename(columns={"Question": "Questions", "Answer": "Answers"})
#importa dataset
conversation_3d_chatbot = pd.read_csv('./datasets/3d-conversation-chatbot.csv')
conversation_3d_chatbot.drop(columns=['Unnamed: 0'], axis=0, inplace=True)
conversation_3d_chatbot = conversation_3d_chatbot.rename(columns={"question": "Questions", "answer": "Answers"})
#importa dataset
# file = open("./datasets/jungleb.txt", "r")
# the_jungle_book = file.read()
# file.close()
# junta datasets
df_concated = pd.concat([mental_health_conversation_dataset, QA_python_dataset, conversation_3d_chatbot], ignore_index=True)
# tira dados vazios
df_concated.dropna()
# reseta indexes
df_concated = df_concated.reset_index(drop=True)
# clona dataset para conservar tratamento
df_clone = copy.deepcopy(df_concated)
# junta colunas
data = {"text": []}
# data['text'].append(the_jungle_book)
for x in range(0, len(df_clone)):
    text = "".join(df_clone['Questions'][x]+" "+df_clone['Answers'][x])
    data['text'].append(text) 
df_merged = pd.DataFrame(data)
df_clone2 = df_merged.copy()
print('done!')

done!


In [69]:
text = df_clone2.text.values
joined_text = "".join(text)
print('done!')

done!


In [82]:
joined_text = joined_text[:500000]

In [74]:
import random

import numpy as np
from nltk.tokenize import RegexpTokenizer

import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop
print('done!')

done!


In [85]:
tokenizer = RegexpTokenizer(r"\w+")
tokens = tokenizer.tokenize(joined_text.lower())
tokens[:100]

['what',
 'does',
 'it',
 'mean',
 'to',
 'have',
 'a',
 'mental',
 'illness',
 'mental',
 'illnesses',
 'are',
 'health',
 'conditions',
 'that',
 'disrupt',
 'a',
 'personâ',
 's',
 'thoughts',
 'emotions',
 'relationships',
 'and',
 'daily',
 'functioning',
 'they',
 'are',
 'associated',
 'with',
 'distress',
 'and',
 'diminished',
 'capacity',
 'to',
 'engage',
 'in',
 'the',
 'ordinary',
 'activities',
 'of',
 'daily',
 'life',
 'mental',
 'illnesses',
 'fall',
 'along',
 'a',
 'continuum',
 'of',
 'severity',
 'some',
 'are',
 'fairly',
 'mild',
 'and',
 'only',
 'interfere',
 'with',
 'some',
 'aspects',
 'of',
 'life',
 'such',
 'as',
 'certain',
 'phobias',
 'on',
 'the',
 'other',
 'end',
 'of',
 'the',
 'spectrum',
 'lie',
 'serious',
 'mental',
 'illnesses',
 'which',
 'result',
 'in',
 'major',
 'functional',
 'impairment',
 'and',
 'interference',
 'with',
 'daily',
 'life',
 'these',
 'include',
 'such',
 'disorders',
 'as',
 'major',
 'depression',
 'schizophrenia',
 '

In [86]:
unique_tokens = np.unique(tokens)
unique_tokens_idx = {"tokens": idx for idx, token in enumerate(unique_tokens)}
unique_tokens_idx

{'tokens': 1530}

In [87]:
# separando dados para treino. pegamos todas as palavras mas 10 palavras a frente
n_words = 10
input_words = []
next_words = []

for i in range(len(tokens)- n_words):
    input_words.append(tokens[i:i+n_words])
    next_words.append(tokens[i+n_words])
print('done!')

done!


In [88]:
# bag of words
# para cada exemplo, pegamos n_words a frente e depois o boolean para cada possível próxima palavra
X = np.zeros((len(input_words), n_words, len(unique_tokens)), dtype=bool)
y = np.zeros((len(next_words), len(unique_tokens)), dtype=bool)

In [89]:
for i, words in enumerate(input_words):
    for j, word in enumerate(words):
        X[i, j, unique_tokens_idx[word]] = 1
    y[i, unique_tokens_idx[next_words[i]]] = 1

KeyError: 'what'

In [80]:
model = tf.keras.models.Sequential()
model.add(LSTM(128, input_shape = (n_words, len(unique_tokens)), return_sequences = True))
model.add(LSTM(128))
model.add(Dense(len(unique_tokens)))
model.add(Activation("softmax"))
print('done!')

done!


In [81]:
model.compile(loss="categorical_crossentropy", optimizer = RMSprop(learning_rate=0.01), metrics = ["accuracy"])
model.fit(X, y, batch_size = 128, epochs = 10, shuffle = True)

Epoch 1/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 8s 74ms/step - accuracy: 0.0707 - loss: 0.0000e+00
Epoch 2/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 5s 76ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 5s 76ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 4/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 5s 76ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 5s 74ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 6/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 5s 77ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 5s 76ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 8/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 5s 74ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 9/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 5s 76ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 10/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 5s 76ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00


In [101]:
# salva um arquivo com o modelo para poder ser recuperado depois
model.save("fourth-try-model.h5")
print('done!')

done!


In [102]:
model = load_model("fourth-try-model.h5")

In [66]:
def predict_next_word(input_text, n_best):
    input_text = input_text.lower()
    X = np.zeros((1, n_words, len(unique_tokens)))
    for i, word in enumerate(input_text.split()):
        X[0, i, unique_tokens_idx[word]] = 1
    predictions = model.predict(X)[0]
    return np.argpartition(predictions, -n_best)[-n_best]
print('done!')

done!


In [97]:
possible = predict_next_word("What have I done", 5)
possible

KeyError: 'what'

In [98]:
def generate_text(input_text, text_length, creativity=3):
    word_sequence = input_text.split()
    current = 0
    for _ in range(text_length):
        sub_sequence = " ".join(tokenizer.tokenize(" ".join(word_sequence).lower())[current:current+n_words])
        try:
            choice = unique_tokens[random.choice(predict_next_word(sub_sequence, creativity))]
        except:
            choice = random.choice(unique_tokens)
        word_sequence.append(choice)
        current += 1
    return " ".join(word_sequence)
print('done!')

done!


In [100]:
generate_text("what can we do about AI ?", 100, 4)

'what can we do about AI ? had get cannot all toolkits idea nit occur includes knows tired future pleasantly limit big agree recommendations tired share inability straightforward playing recommendations opportunity training therapy reasons spaces severe common untrue delivery 123 effects seems mad hand worked engage cured foreseeable cases months service neurological loneliness unhealthy cards telling avoid actually wouldn addition disrupted includes variety in museums reactions nutrition make performances successful strong member harvard topic either fits impairment physical bc211 behavioral comes vegetables which social management fresh we childâ informed browsing believing research never deal 6 3 sexual disorder pancreas families avoided groceries set pass sick schizophrenia build'

In [ ]:
# As palavras não são encontradas na bag of words

In [ ]:
# Da respostas completamente desconexas mas acho q isso acontece pois a func predict_next_word não funciona corretamente